In [4]:
import ee
import json
import requests
import geopandas as gpd

In [6]:
# Authenticate
#ee.Authenticate()

# Initialize API
ee.Initialize()

In [2]:
def get_rgb(aoi_path,start_date,end_date):
    #Cargar area de interes
    with open(aoi_path) as file:
        aoi_json = json.load(file)
    aoi = ee.Geometry(aoi_json['features'][0]['geometry'])
    #Obtener coleccion de imagenes dentro de las fechas
    sentinel2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterDate(start_date, end_date).filterBounds(aoi)
    #Descargar raster RGB
    for image in sentinel2.getInfo()['features']:
        # Obtiene el id y la fecha de la imagen
        date = ee.Date(image['properties']['system:time_start']).format('YYYY-MM-dd').getInfo()

        # Crea el nombre del archivo
        filename = f"{date}.tif"

        # Selecciona la imagen y recorta con el área de interés
        img = ee.Image(image['id']).select(['B4', 'B3', 'B2']).clip(aoi)

        # Descarga la imagen
        url = img.getDownloadUrl({
            'name': filename,
            'scale': 10,  # escala en metros
            'crs': 'EPSG:4326',  # sistema de coordenadas
            'format': 'GEO_TIFF'
        })

        # Descarga el archivo
        response = requests.get(url)
        with open(filename, 'wb') as f:
            f.write(response.content)

In [7]:
aoi_path = '/home/alejandro/Documentos/GIS/El_Pilar/Area_Interes.geojson'
start_date = '2024-07-28'
end_date = '2024-08-09'
get_rgb(aoi_path,start_date,end_date)